<span type="title">MySQL学习指南</span> | <span type="update">2018-05-31</span> | <span type="version">1.10</span>

<span type="intro"><p type="card-text">本部分主要介绍MySQL的基础使用。分别讲解了表的创建、修改、恢复；数据的添加和删除；数据的查询；数据的更新和删除；连接和子查询；字符串、日期函数；数据库基础管理；数据库的备份和恢复。在最后介绍了在Windows和Linux下部署MySQL的一些问题。</p></span>

# 1. MySQL 表的创建和修改

## 1.1 表的查看、删除和添加

**在这一部分，将重点了解 `CERATE TABLE`, `SHOW`, `DROP TABLE`, `MYSQL DUMP`, `ALTER TABLE`, `RENAME TABLE` 的用法。**

**这些命令都是针对表格和列级别的，不涉及数据行。**

使用 `create database dbname` 创建数据库。使用 `drop database dbname` 删除数据库。

使用 `show database;` 查看数据库， 使用 `use database` 切换数据库。 使用 `show tables;` 显示此数据库的所有表。

### 1.1.1 创建表 `CREATE TABLE`

MySql创建表使用以下命令：

```mysql
CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
    [(create_definition,...)]
    [table_options]
    [partition_options]
    [IGNORE | REPLACE]
    [AS] query_expression
```

下面是一个例子：

```mysql
create table [dbname.]tablename 
    (
    column_name describe_things,
    column_name describe_things,
    column_name describe_things
    
    primary key (column_name),
    unique key key_name (column_name)
    )
    [engine = InnoDB/MyISAM default charset=utf8]
    ;
```

其中常见的数据类型有：`int varchar(length) blob text char(l) bit enum('A','B',..) date`

常见的附属设置有： `primary key, auto_increment, unique`

`not null default`

（auto_increment 必须和 primary key 一起使用）



其中，varchar表示可变长字符，可节省内存，一定要定义长度。主键和唯一键可以写在列名后，也可以单独写一行，放在括号内，引擎和编码放在括号外。

注意，括号里面最后一行一定不要加逗号。

### 1.1.2 查看表 `SHOW`

创建表之后可以有两种方式查看所创建的表格结构：

```mysql
describe table tbname;
show create table tbname;
show column from tbname [like colname];
show index from tbname [where Key_name = 'ss']
```

其中，describe 命令用来显示表结构，后缀为 \g 或者 \G 可以切换显示方式。

show create table 命令用来以一种更紧凑方式显示表结构，可由这些语句重复创建相同结构的表。

show column 可以以更细的精度查看各列属性，可以选择某列显示，注意这里使用的是 like 字段。而对于 show index， 使用的是 where 字段。

其中 show 语法和常用的 show 命令如下：

```mysql
SHOW [EXTENDED] [FULL] {COLUMNS | FIELDS}
    {FROM | IN} tbl_name
    [{FROM | IN} db_name]
    [LIKE 'pattern' | WHERE expr]
    
SHOW DATABASES [like_or_where];
SHOW CREATE DATABASE db_name;
SHOW TABLE STATUS [FROM db_name] [like_or_where];

SHOW CREATE TABLE tbl_name;
SHOW INDEX FROM tbl_name [FROM db_name];

SHOW CHARACTER SET [like_or_where];
SHOW WARNINGS [LIMIT [offset,] row_count]
```

### 1.1.3 删除表和添加数据 `DROP TABLE`

使用 `drop table tbname` 删除表格。

使用 `insert into` 命令添加数据。

```mysql
insert into [dbname.]tbname (col_1,col_2) 
values (col_1r1,col2_r1),(col_1r2,col2_r2),
(col_1r3,col2_r3);

```

## 1.2 表的修改、备份与恢复

### 1.2.1 备份和恢复 `MYSQL DUMP`

在更改表之间，一定要备份数据，使用以下命令：

`mysqldump -u username -p dbname [tbname] > path/filename.sql`

恢复数据使用：

`mysql -u username -p dbname [tbname] < path/filename.sql`

恢复数据会删除现有的数据库/表，然后重新写入。

### 1.2.2 将表复制到它处 `CREATE TABLE`

一般来说，当更改表的时候，一般将需要更改的表复制到tmp数据库，然后更改并检查后进行替换。可以这样做：



```mysql
CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
    { LIKE old_tbl_name | (LIKE old_tbl_name) }
//such as
create table dbname.tbname like dbname.tbname_old;
insert into dbname.tbname select * from dbname.tbname_old;
```

这种方法的第一句话使用 like 字段复制了整个表的表结构，第二句话选了全部数据进行插入，注意这里可以选择部分数据插入的，然后清理多余的表结构即可。

第二种方法是，直接复制表结构和数据，不过这样会丢失一些信息，比如主键和递增，后期需要处理。

```mysql
create table dbname.tbname select * from dbname.tbname_old
```

### 1.2.3 修改表的结构 `ALTER TABLE`

使用 `alter table` 进行表结构的修改。

```mysql
alter table dbname.tbnam
add column col_1 col_info first，
add column col_2 col_info after|before col_1,
drop column col_1,
change column col_name_old col_name_new new_column_info
modify column col_name enum('A','B','C') after col_1;
```

如果某一句出错，全部都不会执行。before 的列的字段必须在alter语句之前就已经有此列。modify 语句和 change 语句不同，不需要写两次列名。

```mysql
alter table tbname
auto_increment = 100
```
这句话可以对表整体进行修改，递增从100开始而不是从1开始。

最后，这里是 alter table 的大多数常用方法：

```mysql
ALTER TABLE tbl_name
    [alter_specification [, alter_specification] ...]
    [partition_options]

alter_specification:
    table_options
  | ADD [COLUMN] col_name column_definition
        [FIRST | AFTER col_name]
  | ADD [COLUMN] (col_name column_definition,...)
  | ADD {INDEX|KEY} [index_name]
        [index_type] (index_col_name,...) [index_option] ...
  | ALTER [COLUMN] col_name {SET DEFAULT literal | DROP DEFAULT}
  | ALTER INDEX index_name {VISIBLE | INVISIBLE}
  | CHANGE [COLUMN] old_col_name new_col_name column_definition
        [FIRST|AFTER col_name]
  | DROP [COLUMN] col_name
  | DROP {INDEX|KEY} index_name
  | DROP PRIMARY KEY
  | MODIFY [COLUMN] col_name column_definition
        [FIRST | AFTER col_name]
  | ORDER BY col_name [, col_name] ...
  | RENAME COLUMN old_col_name TO new_col_name
  | RENAME {INDEX|KEY} old_index_name TO new_index_name
  | RENAME [TO|AS] new_tbl_name
  | {WITHOUT|WITH} VALIDATION

```

### 1.2.4 表的重写 `RENAME TABLE`

当处理好表的结构后，使用

```mysql
rename table dbname.tbname to dbname.tbname_new，
    [dbname2.tbname2 to dbname2.tbname2_new];
```

需要注意,和 change column 不同，这里有一个 to 字段。

注意这里的 rename 和 alter table rename 的区别，前者处理表，后者处理列。

### 1.2.5 索引创建

使用 add index 添加索引，比如，下列语句创建了一个包含姓名的复合索引，叫做 human_names , 这样就可以使用下列语句来精确查找了。

`show index from x where Key_name = human_names` 

```mysql
alter table dbname.tbname
add index human_names (name_first,name_last);
```

<br>
<hr>

# 2. 添加和删除数据

**在这一部分，将重点了解 `INSERT,` `REPLACE`, `DELETE`的用法**

对于数据库，遵循这样一个原则，没有数据的时候，不要建表，当创建主表后，再去创建其余表格。因为开发是一个过程，没有办法预知之后需要多少表格，表格的结构是什么样的。

## 2.1 添加数据行 `INSERT`

mysql 向一个已经有列结构的表中添加数据一般使用下面语句：

```mysql

INSERT [LOW_PRIORITY | DELAYED | HIGH_PRIORITY] [IGNORE]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]  //such as select * from xxx
    [(col_name [, col_name] ...)] //such as (t1,t3)
    {VALUES | VALUE} (value_list) [, (value_list)] ... //such as values (t11,t12) (t21,t22)
    [ON DUPLICATE KEY UPDATE assignment_list]

INSERT [LOW_PRIORITY | DELAYED | HIGH_PRIORITY] [IGNORE]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    SET assignment_list
    [ON DUPLICATE KEY UPDATE assignment_list]
```

**标准插入方法**

insert 语句可以这样写，但是要求是，对于那些不想要填写的列，需要使用 NULL 表示，否则会出错。

```mysql
insert into tbname 
values (col1, col2, col3, NULL, col4)
```

或者也可以这样写，直接制定某些列，对于其余列，递增就递增，否则为默认值：

```mysql
insert into tbname (col1_name, col2_name, col4_name)
values (col1, col2, col4),(col1_2, col2_2, col4_2)
```

**扩展插入方法**

还有第三种写法，这似乎是mysql的扩展而非标准 SQL 语法：

```mysql
insert into tbname
set col1_name = "col1_row_1_value", col2_name = "col2_row2_value"
```

**避免错误中断**

如果每次输入多行数据，其中有一行有问题，使用 ignore 可跳过错误行而不影响其余行。

```mysql
insert ignore into tbname (col1, col2)
values (col1_1, col2_1), (col1_2, col2_2);
```

**设置优先级**

对于生产环境的机器，进行大量数据插入会降低读取能力，因此可选优先级以解决这个问题。对于InnoDB，其在插入时会锁定改行，而对于其余引擎，则会锁定改表以阻塞读取过程，对于非InnoDB，设置优先级才有用。

**注意：**

ON DUPLICATE KEY处理重复数据、PARTITION 从 select 语句直接插入见第四部分`UPDATE`使用方法。

<br>


## 2.2 替换数据行 `REPLACE`

默认的insert会重复插入数据，使用 `replace into tbname (col1, col2) values (col1_1, col1_2)`  替换已有数据。替换的标准（程序会检测自动递增的列、主键和唯一键来确认是否重复），如果不重复就像 insert 一样工作，否则删除后再 insert。如果要更改行，那么使用 update 更好。

```mysql
replace into bird_families (scientific_name, brief_description, order_id)
values 
('Viduidae2','Indigobirds & Whydahs',1128),
('Estrildidae2','Waxbills, Weaver Finches, & Allies',1128),
('Ploceidae2','Weavers, Malimbe, & Biships',1128);
```

replace 的方法如下：

```mysql
REPLACE [LOW_PRIORITY | DELAYED]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [(col_name [, col_name] ...)]
    {VALUES | VALUE} (value_list) [, (value_list)] ...

REPLACE [LOW_PRIORITY | DELAYED]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    SET assignment_list

REPLACE [LOW_PRIORITY | DELAYED]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [(col_name [, col_name] ...)]
    SELECT ...
```

## 2.3 删除数据行 `DELETE`

对于输入的内容，如果想要删除，即删除某些/全部行，使用 delete from 即可。

```mysql
delete from tbname where id = '1221';

DELETE [LOW_PRIORITY] [QUICK] [IGNORE] FROM tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]
```

## 2.4 展示数据

**使用 like 过滤字段名**

此外，使用 % 通配符可以过滤那些含有特定字符的列：

```mysql
show column from tb1 like "%_id";
```

这段话选中了所有 tb1 中以 _id 结尾的列。


**使用 order 和 limit 排序并且过滤选取**

当插入很多数据后，我们必然有查看数据和列的需求，采用 order by 的 select 可以很好的解决这个问题。

select ... from ... 的用法如下：

```mysql
select colname1, colname2 
[from tbname]
[where where_condition]
[order by colname [ASC|DESC]]
[limit row_count]
```

比如，选取名称和 id，然后根据 id 降序排列，输出第一个：

`select id,name from tb1 order by id desc limit 1`



**聚合输出，使用 as 作为新的列名**

最后，要聚合输出结果，使用 as 来设置新的列名，下面是一个例子。

```mysql
select  birds.common_name as 'Bird',
        birds.scientific_name as 'Scientific_name',
        bird_families.scientific_name as 'Family',
        bird_orders.scientific_name as 'Order' 
from    birds, bird_families, bird_orders 
where   birds.family_id = bird_families.family_id 
        and bird_families.order_id = bird_orders.order_id;

```

这里非常容易出错的一点是，如果你使用 notepad++ 这种软件写好语句，然后复制到 shell 中执行，需要确认是否换行被完全删掉，比如上面这句话复制到 shell 有以下问题：as 'Order'from birds,... 这里的 from 前没空格，造成语法错误。

关于查询更详细的部分见下一节。

<hr>
<br>

# 3. 查询数据 `SELECT`

**在这一部分，将重点了解 `SELECT` 的用法**

查询数据主要使用 `select` 语句，其如下所示：

```mysql
SELECT
    select_expr [, select_expr ...]
    [FROM table_references
      [PARTITION partition_list]
    [WHERE where_condition]
    [GROUP BY {col_name | expr | position}
      [ASC | DESC], ... [WITH ROLLUP]]
    [ORDER BY {col_name | expr | position}
      [ASC | DESC], ...]
    [LIMIT {[offset,] row_count | row_count OFFSET offset}]
```

在上面简要介绍了大致的用法，这里的介绍则更加深入。

**where 字段**

对于 `where` 字段，一般使用 = in 等运算符来进行判断，比如：

`where family_id in (12,13,15)` 或者 `where family_id = 14`

对于不等，可以使用：

`where common_name != ''` 或者 `where common_name <> ''`

对于多个判断字段，使用 and 将其连接起来即可。

对于模糊搜索，可以使用 like 或者 regexp 语法。

**where字段：like 语法**

`where common_name like 'Least%'` 选中以Least开头的名字，百分号通配。其中有可能有binary 字段的区别。

**where字段：regexp 语法**

regexp 正则表达式能更加强大的选择文本，比如 `where common_name regexp 'Least|Great' `选中包含Least或者Great的名字。

**select/from字段：as 语法**

可以为列和表起别名，使用 as xxx 即可。需要注意的是，如果设置了别名，在之后的代码中，只能够使用别名进行指代，而不能使用原来的名字。

`select common_name as 'Bird Name' from bird_order as 'order'`

**limit 字段**

使用 limit 进行输出限制， `limit 2,3` 代表从第3行开始，输出3行的内容，0为第1行。

下面是一个简短的例子，使用到了表连接，别名，模糊查询，排序和限制

```mysql
select common_name as 'Bird',
families.scientific_name as 'Family',
orders.scientific_name as 'Order'
from birds, bird_families as families, bird_orders as orders
where birds.family_id = families.family_id
and families.order_id = orders.order_id
and common_name like 'Little%'
order by 'Bird' 
limit 0,5;
```
一个正则表达式的例子，可以使用 regexp 以及 not、binary 字段：

```mysql
select common_name from birds
where common_name regexp 'ABBOTT|Abert'
and common_name not regexp 'Booby' 
and common_name regexp binary 'Abert'
```

**select_expr 字段：count 语法**

count 语法一般用来进行计数，比如这样：

```mysql
select count(*) from birds;

+----------+
| count(*) |
+----------+
|    28891 |
+----------+

select count(description) from birds;

select  families.scientific_name as 'Family',
        count(*) as 'Number of Birds',
        count(common_name) as 'Birds have common Name'
from birds, bird_families as families
where birds.family_id = families.family_id
and families.scientific_name = 'Ploceidae';

+-----------+-----------------+------------------------+
| Family    | Number of Birds | Birds have common Name |
+-----------+-----------------+------------------------+
| Ploceidae |               5 |                      5 |
+-----------+-----------------+------------------------+

select  families.scientific_name as 'Family'
from birds, bird_families as families
where birds.family_id = families.family_id
and families.scientific_name = 'Ploceidae';

+-----------+
| Family    |
+-----------+
| Ploceidae |
| Ploceidae |
| Ploceidae |
| Ploceidae |
| Ploceidae |
+-----------+
```

其中，对于某列计数会忽略此列的NULL值。可以在 set 语句使用 count(col_name) 来对项目进行计数。**需要注意的是，这不是聚合或者分组，生成的结果只有一行，可以看做先查询再求count()函数。**比如在上面的scientific_name列中含有100行，然后count(*)就会是100。

# 4. 更新和删除数据 `UPDATE` , `DELETE..USING`

## 4.1 更新数据

`UPDATE` 语句的使用方法如下：

```mysql
UPDATE [LOW_PRIORITY] [IGNORE] table_reference
    SET assignment_list
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]

```

其中，和 `INSERT INTO` 很像，有 `SET` 字段，但是没有 `INTO` 字段，也没有 `CREATE TABLE` 的 `TABLE` 字段。

除此之外，可以多表共同更新，但是当多表更新的时候，不能使用 `ORDER BY`， 只能够使用子查询。

<u>需要注意，`SET` 字段中可以写多个条件，使用逗号连接，而不是使用多个 `SET` 字段，或者使用 `WHERE` 字段中多个 `AND` 那样。</u>

比如：`set name_first = "Corkine",name_last = "Ma"`

**基本流程**

```mysql
select human_id, name_first, name_last 
from humans 
where name_first = 'Rusty' 
and name_last = 'Osborne';

//find human_id is 3 and unique and do that:

update humans 
set name_last = 'Johnson' 
where human_id = 3;

//之后再次确认信息：

select name_last, name_first, human_id 
from humans 
where human_id = 3;

```

基本流程指的是，更新可能影响数据，因此最好先查询，之后确认要更新的列，更新后再查询，这样一套流程。

**涉及到列结构的流程**

有些更新可能涉及列结构的变化，一般有两种方法，其一，先更改数据，然后再处理列结构。其二，先扩展列结构，然后修改数据后，再修改列结构。

比如：

```mysql
现在，比如说想要更改称呼，先查看目前的列结构

show column from humans like "formal_title"\G

第一种方法是先处理目前数据，然后再更改列结构。

update humans 
set formal_title = 'Ms.' 
where formal_title in('Miss','Mrs.');

alter table humans 
change column formal_title formal_title enum('Mr.','Ms.');

第二种方法是先扩展表结构，然后再更改数据,再更改表结构

alter table humans 
change column formal_title formal_title enum('Mr.','Ms.','Mr','Ms');

update humans 
set formal_title = substring(formal_title, 1, 2);

alter table humans 
change column formal_title formal_title enum('Mr','Ms');
```

**随机抽取、排序和限制**

似乎排序和限制在 update 中没有什么作用，但这种看法是错误的，因为，考虑下面这个例子：想要随机抽取两人并标记：

```mysql
1、创建一个表
create table prize_winner 
(winner_id int auto_increment primary key,
human_id int,
winner_data date,
prize_chosen varchar(255),
prize_sent date);

虽然说，每个人只有一次机会中奖，
但是，对于每个表建立一个专属的id列，是一种健壮的设计

2、插入数据
insert into prize_winners (human_id) 
select human_id form humans;

3、抽奖开始
update prize_winners 
set winner_date = curdate() 
where winner_date is null 
order by rand() 
limit 2;

curdate() 返回今日日期
date('2018-11-12') 构造日期 
rand() 生成随机数并随机选两个人发奖品；
```

**有条件的多表更新**

`UPDATE` 可以使用多个表来进行更新，但是不能在这样的语句中使用 `ORDER BY`， 因此，可以使用子语句，如下：

```mysql
1、错误示例
update prize_winners, humans 
set winner_date = curdate() 
where winner_date is null 
and country_id = 'cn' 
and prize_winners.human_id = humans.human_id 
order by rand() 
limit 2;

2、正确的例子
update prize_winners 
set winner_date = curdate() 
where winner_date is null 
and human_id in (select human_id from humans where country_id = 'cn' order by rand())
limit 2;
```

## 4.2 重复数据标记 `INSERT ON DUPLICATE KEY UPPDATE`

在上一部分，讲过使用 `DELETE` 语句，当时，有一部分没讲，就是 `ON DUPLICATE KEY UPDATE` 这一句，如果我们想要把一堆数据插入另一堆数据中，这两组数据有可能重复，而我不希望使用 `REPLACE` 进行替换，而是希望保留和标记这些数据，可以这样做：

```mysql
1. 创建新列放置这些标记
alter table humans 
add column better_birds_site tinyint default 0;

2. 使用带有 UPDATE 的 INSERT 语句
insert into humans 
(formal_title, name_first, name_last, email_address, better_birds_site) 
values ('Mr','Barry','Pilson','barry@gomail.com',1),
('Ms','Lexi','Hollar','alexandra@mysql.com',1),
('Mr','Ricky','Adams','ricky@gomail.com',1) 
on duplicate key 
update better_birds_site = 2;
```

## 4.3 删除多个相关表的数据 `DELETE .. USING`

很多时候，对于删除，需要使用多个表联合来删除数据，否则要对于每张表逐个删除，比如现在有两张名为 humans 和 prize_winners 的表，需要删除一个用户，这个用户信息在两张表都存在，那么就需要用到 USEING 字段，如下：

```mysql
delete from humans, prize_winners 
using humans join prize_winners 
where humans.name_first = 'Liu'
and humans.name_last = 'Jin' 
and humans.human_id = prize_winners.human_id
```

# 5. 连接和子查询

## 5.1 UNION 语句

```mysql
select *** from *** where *** 
union
select *** frin *** where ***

select repeat('a',1) union select curtime();
+---------------+
| repeat('a',1) |
+---------------+
| a             |
| 20:43:56      |
+---------------+
```


如上所示，连接可以连接多条输出，将其整合到一个表中，方便查看。如果输出的信息不同，那么以第一次选择的信息作为列标题。

**UNION 字段只能出现在多个 SELECT 语句中**

## 5.2 JOIN 语句

**`JOIN` 的两种写法**

在之前的语句中，我们使用以下方法选择多个表的信息：

```mysql
select book_id, orders.order_id
from books, orders
where books.book_id = orders.book_id
```

下面是一种替代方案：

```mysql
select book_id, orders.order_id
from books join orders on(books.book_id = orders.book_id)
```
这样写的好处在于，可以避免 where 语句的检测失效，因为在原来版本中，如果 where 子语句中条件不成立，那么什么都不会返回，而使用 join 可以避免这一问题，如果无法连接就会报错。

如果需要连接的列名称一样，甚至可以这样做：

```mysql
select book_id, orders.order_id
from books join orders using(book_id)
```

**`LEFT[RIGHT] JOIN`**

如果 `A JOIN B`, 它建立多个表之间的关系，使用给定的键，如果A表（主表）id为1的有2条，B表id为1的记录有1条，那么建立的JOIN含有2条记录，查询A表时，这两条记录都会使用B表中相同的信息。如果B表中没有出现A表中的信息，那么会删除连接后的A表的这些字段。

如果 `A LEFT JOIN B`，那么以A表为主，B表中没有的A表记录在查询B表信息的时候会被标记为NULL，这些行不会被直接删除。而查询A表原本信息时，则不受任何影响。

如果 `A RIGHT JOIN B`，那么以B表为主，在连接后，如果查询联合表中的A表字段，而恰好B表中有，而A表中没有，那么就会返回NULL。而查询B表原本信息时，则不受任何影响。

**`DELETE & JOIN`**

DELETE 语句中也有 JOIN 语句，但是不能写成： `DELETE FROM tabl join tab2 using(col1)` 这种样子，这是固定的语法，不能修改。

下面是一个例子，之前的方案不太好，因为万一 where 里不匹配或者出现错误，就没有任何更改发生：

```mysql
delete from humans, prize_winners 
using humans left join prize_winners using(human_id)
where name_first = 'Elena' and name_last = 'Bokova';
```

DELETE 在 JOIN 语句 上的应用主要是，它可以用来清除那些多余的数据，比如我们的姓名库中没有了`'Liu Jin'`这个人，而 `prize_winner` 中还有，可以用下面语句进行删除：

```mysql
select humans.human_id 
from humans right join prize_winners using(human_id);

delete from prize_winners 
using humans right join prize_winners using(human_id)
where humans.human_id is null;
```

**`UPDATE & JOIN`**

下面是一个例子，展示了使用更新语句如何选择多表，并且为 `RALLIDAE` 科为空白的状态更改为1的。注意，这个例子中使用了 `LEFT JOIN conservation_status` 的目的是想要选择那些为空的列，而在 `bird_families` 表则没有这样做，因为没有必要。

```mysql
update birds 
left join conservation_status using(conservation_status_id) 
join bird_families using(family_id) 
set birds.conservation_status_id = 1 
where bird_families.scientific_name = 'Rallidae' 
and conservation_status.conservation_status_id is null;
```

## 5.3 子查询

子查询指的是被放置在外部查询的 `from` 或者 `where` 中的查询，它们返回必要的值或者数据序列，供外部查询调用。子查询总是先于外部查询发生。子查询必须返回外部查询所需要的类型的数据。

**标量子查询**

顾名思义，这种类型的子查询返回的是一个标量，一般用在 `where` 语句的 `=` 之后。

```mysql
select scientific_name as Family 
from bird_families 
where order_id = 
(select order_id from bird_orders where scientific_name = 'Gaviiformes')
```
一般情况下，会对想要变成子查询的查询设置 `limit 1` 的限制，以确保其返回的值为标量而非序列。

**列子查询**

列子查询多用于以下情况(IN 运算符判断下)：

`select xxx in(select xx from xx where xx order by xx)`


**行和表子查询**

对于表子查询，其必须用圆括号括起来，并且返回一个别名，一般跟在 `from` 后面。

```mysql
select family as 'Bird Family',
count(*) as 'Number of Birds' 
from 
(select families.scientific_name as family from birds join bird_families as families using(family_id) 
where families.scientific_name is not null) as derived_1 
group by family;

+------------------+-----------------+
| Bird Family      | Number of Birds |
+------------------+-----------------+
| Anseriformes     |             119 |
| Charadriiformes  |             207 |
| Estrildidae      |               4 |
| Gaviidae         |               6 |
| Ploceidae        |               5 |
| Podicipediformes |             223 |
| Rallidae         |              92 |
| Viduidae         |               3 |
+------------------+-----------------+
```

这里有一点需要注意，如果不使用 `GROUP BY` 语句，那么结果为：

```

+--------------+-----------------+
| Bird Family  | Number of Birds |
+--------------+-----------------+
| Anseriformes |             659 |
+--------------+-----------------+
```
可以认为，count(*) 函数的用法就是对于最后生成的表进行计数，如果返回的是一个 groupby 对象，则分别计数。

# 6. 字符串函数

用于字符串的函数有很多，这里主要介绍一些常用的。

**`CHAR_LENGTH(str), LENGTH(str), BIT_LENGHT(str)`**

前者返回char长度，abc占三个字符，你好吗占3个字符。

中间返回byte长度，abc占3个byte，你好吗占6个byte。

最后返回bit长度，不同实现可能不同。

```mysql
char_length('abc') -- 2
length('abc') -- 2
char_length('你好'); -- 2 字符
length('你好'); -- 4 字节
bit_length('你好') -- 32 位数

-- 根据字符长度进行判断
select if(char_length('corkine')>3,'a','b'); --a
-- 删除有奇怪字符的行
delete from birds where char_length(comments) != length(comments);
```

## 6.1 单字符串函数

**常见的操作有 大小写转换`LCASE, RCASE`、类型转换`CONVERT, CAST`、文本精简`LTRIM, RTRIM, TRIM`、文本填充`LPAD, RPAD`**


**`L[R]case(str)、QUOTE(str)`**

大小写转换；添加引号和反转义序列，防止SQL注入。

```mysql
select quote(concat(formal_title,'.',lcase(name_first))) as A,
ucase(name_last) as B 
from humans;
```

**`CONVERT(expr,type), CONVERT(expr USING transcoding_name)`**

用来进行类型转换，比如：

```mysql
select convert('0123',DECIMAL); -- 123
```

常见的类型有：`nchar(n) signed unsigned time integer json decimal char(n) date
datetime binary`

`CONVERT('0123',DECIMAL)` 和 `CAST('0123' as DECIMAL)` 作用一样。


**`L[R]TRIM(str)`**

`TRIM([{BOTH | LEADING | TRAILING} [remstr] FROM] str), TRIM([remstr FROM] str)`

去除左边，右边，两头的空格或者其他字符，如果不写，默认为空格。

```mysql
SELECT TRIM('  bar   ');
-> 'bar'
SELECT TRIM(LEADING 'x' FROM 'xxxbarxxx');
-> 'barxxx'
SELECT TRIM(BOTH 'x' FROM 'xxxbarxxx');
-> 'bar'
SELECT TRIM(TRAILING 'xyz' FROM 'barxxyz');
-> 'barx'
```

**`L[R]PAD(str,len,padstr)`**

填充空白字符，输入字符总长度，如果小于目标字符，则截取。

```mysql
SELECT LPAD('hi',1,'??');
-> 'h'
SELECT LPAD('hi',4,'??');
-> '??hi'
```

## 6.2 字符串、字符和索引

**你可以查找字符串中的字符，返回索引，使用 `SUBSTRING_IDNEX, LOCATE, FIND_IN_SET`。你也可以使用索引查找字符串中的字符（组），使用 `SUBSTRING, MID, LEFT, RIGHT`。**

**`LEFT()/RIGHT()/MID(str,pos,len)`**

这些函数用来根据索引截取字符串。从第 pos 位开始，长度为 len.

```mysql
select 
left(common_name,3) as A,
mid(common_name,4,10),
right(common_name,15) 
from birds limit 4;
```

MID() 本质上是 SUBSTRING() 的别名。

**`SUBSTRING()`**

SUBSTRING() 有四种写法：

- `SUBSTRING(str,pos)`

- `SUBSTRING(str FROM pos)`

- `SUBSTRING(str,pos,len)`

- `SUBSTRING(str FROM pos FOR len)`

```mysql
select substring('corkine',3,2) -- rk 第三位开始，截取两位
select substring('corkine' from 3 for 2) -- rk 同上
select substring('corkine' -3,2) -- in 
select substring('corkine' from -3 for 2) -- in
select substring('corkine', 3) -- rkine
```

**`SUBSTRING_INDEX(str,delim,count)`**

这个工具的目的是按照固定的分隔符截取字符串，而不是SUBSTRING按照索引位置截取。

substring_index 取已经遍历过的部分，一直到目标点的符号位置。

```mysql
select substring_index('www.abc.com','.',2) -- www.abc
select substring_index('www.abc.com','.',1) -- www
select substring_index('www.abc.com','.',-1) -- com
select 
substring_index(substring_index('www.abc.com','.',2),'.',-1) -- abc
```

**`LOCATE(substr,str), LOCATE(substr,str,pos)`**

找到子字符串所在的索引位置，有限搜索。

```mysql
select locate('co','corkinecor')  -- 只找到第一个
select locate('co','corkinecor',5) -- 只找到第二个，5表示从第五个字符开始
select substring('corkine-ma',locate('-','corkine-ma')+1); -- ma
-- 这个例子也可以使用 substring_index('corkine-ma','-')搜索
```

**`FIND_IN_SET(str,strlist)`**

这个函数用来在序列中寻找值，返回索引。

```mysql
select find_in_set('a','a,b,c,a'); -- 1 只返回1而不返回4
```



## 6.3 多字符串函数

**常用的操作有：替换`REPLACE`、插入`INSERT`、比较`STRCMP`、连接`CONCAT`。**

**`REPLACE(str,from_str,to_str)`**

替换字符串。

```mysql
select replace('www.baidu.com','www','http://www');
-- http://www.baidu.com                
```

**`INSERT(str,pos,len,newstr)`**

插入新字符串到老字符串某个位置， pos 表示从此位置插入，len 表示新字符串复写长度。

```mysql
select insert('abc@abc.cn',4,0,'_duplicate');
-- abc_duplicate@abc.cn  

select insert('abc@abc.cn',4,4,'_duplicate');
-- abc_duplicate.cn  
```

**`STRCMP(expr1,expr2)`**

如果相同，则返回0，反之可能返回1或者-1，使用abs()返回1。

```mysql
select strcmp('cm@muninn.cn','cm@muninn.cn');

-- 用处在于：

insert into possible_duplicate_email (huaman_id,email_1,email_2) 
values (2,'cm@muninn.cn','cm@muninn.com') 
where abs(strcmp('cm@muninn.cn','cm@muninn.com')) = 1;
```

**`CONCAT(str1,str2,...)`**

拼合字符串： 

```mysql
select concat(formal_title,'.',name_first,space(1),name_last) as Birder,
concat(birds.common_name,'-',birds.scientific_name) as Bird 
from bird_sightings join humans using(human_id) 
join birds using(bird_id);
```

**`CONCAT_WS(separator,str1,str2,...)`**

以特定间隔拼合字符串

```mysql
mysql -u xxxxxx -p -h x.muninn.cn --skip-column-names -e \
"select concat_ws('|',formal_title,name_first,name_last 
) from cmdb.humans;" > output_data.log
```

**`IFNULL()`**

还是上面的例子，这种写法可以避免格式问题：

```mysql
"select concat_ws('|',ifnull(formal_title,''),
ifnull(name_first,''),ifnull(name_last,'')) from cmdb.humans;"
```

# 7. 日期、时间、数值和聚合函数

## 7.1 日期和时间函数

**日期类型**

MYSQL 中的日期类型有以下五种：

- `date` yyyy-mm-dd

- `time` 000:00:00

- `datetime` yyyy-mm-dd hh:mm:ss

- `timestamp` 23123211(只能够表示1970-2038年的数值)

- `year` 

**获取日期和时间**

以下这几个返回的都是 datetime 类型的数据。

```mysql
now()
current_timestamp()
localtime()
localtimestamp()
sysdate() -- 返回执行语句前的系统时间
select now(),sleep(2),sysdate(),sleep(2),sysdate();
```
```
+---------------------+----------+---------------------+----------+---------------------+
| now()               | sleep(2) | sysdate()           | sleep(2) | sysdate()           |
+---------------------+----------+---------------------+----------+---------------------+
| 2018-05-29 08:57:58 |        0 | 2018-05-29 08:58:00 |        0 | 2018-05-29 08:58:02 |
+---------------------+----------+---------------------+----------+---------------------+
```

以下语句返回其余格式时间/日期：

```mysql
curdate()
curtime()
unix_tempstamp()
select now(), curdate(), curtime(), unix_tempstamp();
```
```
+---------------------+------------+-----------+------------------+
| now()               | curdate()  | curtime() | unix_timestamp() |
+---------------------+------------+-----------+------------------+
| 2018-05-29 08:59:22 | 2018-05-29 | 08:59:22  |       1527555562 |
+---------------------+------------+-----------+------------------+
```

**数据转换**

你可以在多种数据上进行转换，比如：

- date() 抽取 date 格式日期

- time() 抽取 time 格式时间

- hour()/minute()/second() 分别抽取小时、分钟和秒，返回 int 格式数值

- monthname() 返回月份的英文 -- May 比如

- dayname() 返回周几，比如 Tuesday

- extract(unit from date) 可以进行一般化转换，其中 unit 指的是字符串，比如'MINUTE','YEAR_MONTH', date 为 date 格式日期。

```
+-------------+-------------+-------------+---------------+---------------+
| date(now()) | time(now()) | hour(now()) | minute(now()) | second(now()) |
+-------------+-------------+-------------+---------------+---------------+
| 2018-05-29  | 09:01:51    |           9 |             1 |            51 |
+-------------+-------------+-------------+---------------+---------------+
```

**格式化日期和时间**

- date_format(date,expr) 根据 expr 格式化 date，然后输出，比如：

    `date_format('2018-11-11 10:00:01',"%h %p") --> 10 AM`
    
- get_format([DATE|TIME|DATETIME],'EUR|ISO|INTERNAL|USA') 根据地方输出不同类型时间日期的 expr

    `date_format('2018-11-12',get_format(DATE,USA)) --> 2018/11/12`
    
- convert_tz(date, 'CST|GMT|...') 根据时区转换日期

    配合 `show variables like 'time_zone|system_time_zone'`更好用
    
**时间和日期段**

- DATE_ADD(date,INTERVAL expr unit)
    
    - 比如： date_add(now(),interval 3 day) -- 三天以后
    
    - adddate() 是 date_add() 的一个别名
    
    
- DATE_SUB(date,INTERVAL expr unit)

unit 和 expr 对照如下表:

```
unit Value	Expected expr Format
MICROSECOND	MICROSECONDS
SECOND	SECONDS
MINUTE	MINUTES
HOUR	HOURS
DAY	DAYS
WEEK	WEEKS
MONTH	MONTHS
QUARTER	QUARTERS
YEAR	YEARS
SECOND_MICROSECOND	'SECONDS.MICROSECONDS'
MINUTE_MICROSECOND	'MINUTES:SECONDS.MICROSECONDS'
MINUTE_SECOND	'MINUTES:SECONDS'
HOUR_MICROSECOND	'HOURS:MINUTES:SECONDS.MICROSECONDS'
HOUR_SECOND	'HOURS:MINUTES:SECONDS'
HOUR_MINUTE	'HOURS:MINUTES'
DAY_MICROSECOND	'DAYS HOURS:MINUTES:SECONDS.MICROSECONDS'
DAY_SECOND	'DAYS HOURS:MINUTES:SECONDS'
DAY_MINUTE	'DAYS HOURS:MINUTES'
DAY_HOUR	'DAYS HOURS'
YEAR_MONTH	'YEARS-MONTHS'
```

注意， `time_to_sec()，sec_to_time()` 可以用来进行时间段和秒数之间的转换。

**时间和日期的比较**

- datediff(date1, date2) 用来比较两个日期的差异，返回天数

- timediff(datetime1, datetime2) 用来比较两个时间的差异，返回时间格式，可通过 time_to_sec() 转化成为秒数。

**一个例子：将日期格式转化成不含0开头的格式**

```mysql
select time_format(time(now()),'%m minute, %s seconds');
 
select concat_ws(', ',if(trim(' 0' from time_format(now(),' %i minute')) = time_format(now(),' %i minute'),concat(trim(' 0' from time_format(now(),' %i minute')),'(s)'),trim(' 0' from time_format(now(),' %i minute'))), trim(' 0' from time_format(now(),' %s seconds'))) as "TIME";

```

```
+----------------------------+
| TIME                       |
+----------------------------+
|  26 minute(s),  21 seconds |
+----------------------------+
```

## 7.2 聚合和数值函数

其中聚合函数较为常用，而数值分析， 如果数据库不大，干脆导出后使用 pandas.DataFrame 分析，如果数据库很大，使用 graphlab.SFrame 分析即可。

**count()**

此函数返回 非 NULL 值的行数，结果是 BIGINT 格式。如果是这种格式 count(*) 则返回所有行，不论是否为NULL。

count()经常和group by一起使用，下面是一个例子：

```mysql
select bird_families.scientific_name as 'Family',
count(*) as 'Number' 
from birds join bird_families using(family_id) 
group by birds.family_id with rollup; -- with rollup 的意思是包含总计行

+------------------+--------+
| Family           | Number |
+------------------+--------+
| Gaviidae         |      6 |
| Anseriformes     |    119 |
| Charadriiformes  |    207 |
| Podicipediformes |    223 |
```

**sum()** 合计信息，所group by的行必须为数字，否则返回NULL。

**avg()** 平均值，同上。

**max()/min()** 最大，最小值，同上。

**group_concat()**

此函数将group中的所有数据合并为一行，用逗号分隔，然后输出。

```mysql
GROUP_CONCAT([DISTINCT] expr [,expr ...]
             [ORDER BY {unsigned_integer | col_name | expr}
                 [ASC | DESC] [,col_name ...]]
             [SEPARATOR str_val])
             
             
select bird_families.scientific_name as 'Family',
group_concat(birds.common_name)  
from birds join bird_families using(family_id) 
group by birds.family_id;

 
*************************** 1. row ***************************
                         Family: Gaviidae
group_concat(birds.common_name): Red-throated Loon,Arctic Loon,Pacific Loon,Yellow-billed Loon,,

```

****

# 8. 数据库基础管理

**一个基本的流程**

```mysql
select User, Host from mysql.user;
-- 先查看当前用户状况

create user corkine@localhost identified by 'passwd'
-- 创建用户，用户名@主机名，注意，不同主机名的相同用户名是不同用户
-- 不使用@表示所有主机均可登陆
-- 注意要设置密码

grant usage on *.* to corkine@localhost identified by 'passwd'
-- 创建权限 usage 到所有表
-- *.* 表示所有表，database.table
-- usage 表示没有权限，all 表示所有一般权限，super 表示特殊权限
-- 可以使用多个 grant 语句创建不同权限/不同表的组合

grant all on cmdb.* to corkine@localhost identified by 'passwd'
-- 先关闭权限，再赋予一定的权限
-- grant insert,update,delete on codb.* for corkine@localhost 可以这样写

show grant for corkine@localhost
-- 查看所有的权限

revoke all on mdb.* from corkine@localhost
-- 撤销所有权限

set password for corkine@localhost = 'passwdnew'
-- 更改密码

rename user corkine@localhost to marvin@localhost
-- 重命名用户名

drop user corkine@localhost;
-- 删除用户
```

# 9. 备份和恢复

## 9.1 备份数据

备份数据库使用 `mysqldump` 工具。

```

mysqldump 

--user 
--password 

--lock-all-tables #对于MYISAM引擎，需要锁住所有表防止更改
--single-transaction #对于INNODB引擎，不需要锁表，但是需要打开此项，提高一致性

--extended-insert 一个表一条insert（dump文件小，容易恢复）
--skip-extended-insert 一行一条（默认）
--complete-insert 加上列名（默认没有列名）

--verbose 显示备份过程中步骤，可以方便查看哪里出错了

--databases db1 db2 db3 #备份多个相互关联的数据库
--all-databases 
--ignore-table =mysql.user 避免导出mysql数据库的密码

> xxx.sql
```

## 9.2 分享数据

分享部分数据采用 `SELECT INTO OUTFILE` 命令,可以选择域的分隔符和换行符。

```mysql
SELECT a,b,a+b INTO OUTFILE '/tmp/result.txt'
  FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'
  LINES TERMINATED BY '\n'
  FROM test_table;
```

这个语句的执行需要特殊权限。

或者采用这种方法：

```mysql
mysql -u xxxxxx -p -h x.muninn.cn --skip-column-names -e \
"select concat_ws('|',formal_title,name_first,name_last 
) from cmdb.humans;" > output_data.log

-- 注意，必须使用双引号，否则不会导出数据
```

## 9.3 导入数据

表的恢复较为繁琐，因为不一定有列的对应，更有甚者，不一定恢复的数据来自于MYSQL，这样的话，要先创建好对应的列，然后向测试表中导入数据，当完全确认后再更改导入的表到生产环境。

导入数据使用 `LOAD DATA INFILE` 命令。

```mysql
load date infile 'path/.csv' into table db.tb fields terminated by ',';

LOAD DATA [LOW_PRIORITY | CONCURRENT] [LOCAL] INFILE 'file_name'
    [REPLACE | IGNORE] -- 忽略错误/替换表
    INTO TABLE tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [CHARACTER SET charset_name]
    [{FIELDS | COLUMNS}
        [TERMINATED BY 'string'] '|' 域以什么结束
        [[OPTIONALLY] ENCLOSED BY 'char'] '"' 每个域放在什么中
        [ESCAPED BY 'char'] '\\' 反斜杠默认，不需要设置
    ]
    [LINES
        [STARTING BY 'string'] 行以什么开头
        [TERMINATED BY 'string'] -- 分隔符 对于windows "\r\n"
    ]
    [IGNORE number {LINES | ROWS}]
    [(col_name_or_user_var
        [, col_name_or_user_var] ...)] 对应列名，如果不想要此列，选择@niente临时变量
    [SET col_name={expr | DEFAULT}, 这里可以对临时变量进行处理
        [, col_name={expr | DEFAULT}] ...]
```

# 附录：Windows下的MySQL服务安装

- 下载MySQL Server 的已编译Windows便携版本 http://dev.mysql.com/downloads/mysql/

- 在系统变量PATH注册 bin 子文件夹

- 在 MySQL 根目录新建 data 文件夹

- 运行 `mysqld --initialize-insecure` MYSQL会初始化程序到 data 文件夹

- mysqld 可以启动服务， mysql 运行 shell，使用 root 登陆，登陆后使用 set password for root@localhost = '' 更改密码以及分配账户

- 命令提示符管理员权限到 bin 子目录下，执行 `mysqld --install Mysql` 以注册服务到 service

- 输入 `net start mysql` 启动服务。

注意，关闭/重启服务使用 `net stop/restart mysql`，可以注册 `mysqld --install-manual Mysql` 来手动启动服务而不是自动（默认）。使用 `mysqld --remove Mysql 从Windows的服务中删除服务项`

## MYSQL 疑难杂症

- OperationalError: (2003, "Can't connect to MySQL server")

更改连接为 '127.0.0.1' 而不是 'localhost'

- OperationalError: (2059, NULL)
    
没输密码，更有可能的情况是，新版本MYSQL更改了加密方式：`caching_sha2_password`

使用 root 权限查看表，如果是新加密方式，则更改为原来的加密方式。

```mysql
select user,plugin from mysql.user ;  
ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';   
```


————————————————————————————

**更新日志**

2018-05-23 SP0.9 添加笔记，完成课后习题

2018-05-24 SP0.9 完成第二部分 添加数据

2018-05-25 SP0.9 完成第二部分 查询数据

2018-05-26 SP0.9 完成第二部分 更新和删除数据

2018-05-27 SP0.8 完成第二部分 表连接和子查询

2018-05-28 SP0.9 完成第三部分 字符串函数

2018-05-29 SP0.8 完成第三部分 日期和聚合函数

2018-05-30 SP0.7 完成第四部分 数据库基本管理